# **LAB 3: Comparative Inference Benchmarking on ARM Server and Edge Devices**

## Introduction

Welcome to Lab 3 in our series on **Optimizing Generative AI Workloads with ARM Processors**! In this lab, we shift our focus to evaluating the performance constraints of generative AI inference between ARM-based server and edge devices. Specifically, we'll use the **OpenELM-3B** model alongside the popular **llama.cpp** framework to benchmark and compare inference characteristics on two different ARM environments: a high-performance Graviton server and a resource-constrained Raspberry Pi.

Throughout this lab, you will:

- **Benchmark Latency Metrics**: Measure and compare the overall inference latency, time-to-first-token (TTFT), and per-token decoding latency across devices.
- **Evaluate Memory Consumption**: Analyze how the OpenELM-3B model's memory footprint varies between an ARM server and an ARM-based edge device.
- **Compare ARM CPU Constraints**: Understand the practical limitations and trade-offs when deploying large language models on different ARM CPUs, from powerful server-class processors to limited edge devices.
- **Utilize llama.cpp Framework**: Leverage the llama.cpp framework for efficient model inference and performance measurement, allowing for cross-device comparisons.

This lab will guide you through setting up the necessary environments, running inference benchmarks on both the Raspberry Pi and Graviton server, and interpreting the results. By the end of the lab, you'll gain insights into:

- How hardware differences affect model performance
- Strategies for optimizing inference on both server and edge ARM devices
- Best practices for deploying large language models across diverse ARM architectures

---

### **Lab Objectives**

1. **Set Up Diverse ARM Environments**  
   Prepare and configure both a Raspberry Pi (edge device) and an ARM Graviton server for inference benchmarking with the OpenELM-3B model and llama.cpp framework.

2. **Benchmark Latency Metrics**  
   Measure key performance indicators, including total inference latency, time-to-first-token (TTFT), and per-token decoding latency, on both devices.

3. **Analyze Memory Consumption**  
   Monitor and compare memory utilization during model inference to assess resource constraints on server and edge devices.

4. **Compare Performance Constraints**  
   Understand the practical implications of deploying large language models on resource-constrained edge devices versus powerful server environments. Identify bottlenecks and optimization opportunities for each scenario.

5. **Document Findings and Insights**  
   Compile your observations, compare the differences in performance, and suggest strategies to mitigate limitations on edge devices while leveraging server capabilities.

---

### **Prerequisites**

- **Basic Understanding of Linux**: Familiarity with Linux command-line operations and system configuration.
- **Experience with C/C++ and Python Programming**: Ability to modify, compile, and execute code across different ARM platforms.
- **Fundamentals of AI Inference**: Knowledge of how inference works in large language models.
- **Completion of Labs 1 and 2 (Recommended)**: Prior experience with ARM intrinsics, quantization, and performance benchmarking for generative AI workloads.

---

**By the end of this lab**, you will have a comprehensive understanding of how ARM server and edge CPUs handle inference for large language models like OpenELM-3B. You will be equipped with the skills to benchmark and analyze latency, time-to-first-token, decoding performance, and memory consumption, enabling you to make informed decisions when deploying AI models across diverse ARM architectures.




## 1. Setting Up the Raspberry Pi for Inference Benchmarking

Before proceeding, ensure you have completed the [Lab 1 setup script](#) on your Raspberry Pi. If not, start by running the setup script:

```bash
./setup_pi5.sh
```

Once the script completes, activate the newly created virtual environment:

```bash
source pi5_env/bin/activate
```

This ensures that all required packages for the lab are preinstalled.

### Building the llama.cpp Inference Framework

If you haven't already, download and build the [llama.cpp](https://github.com/ggerganov/llama.cpp) inference framework with ARM CPU optimizations:

1. **Clone the repository:**

   ```bash
   git clone https://github.com/ggerganov/llama.cpp
   ```

2. **Configure the build:**

   ```bash
   cmake -B llama.cpp/build -S llama.cpp -DCMAKE_CXX_FLAGS="-mcpu=native" -DCMAKE_C_FLAGS="-mcpu=native"
   ```

3. **Compile the framework:**

   ```bash
   cmake --build llama.cpp/build --config Release -j$(nproc) -v
   ```

These commands build the framework optimized for your ARM CPU.

### Downloading the OpenELM-3B Model Weights

Next, download the large language model weights from Hugging Face. Follow these steps:

1. Create a Python file named `download_openelm.py` in the root directory with the following content:

   ```python
   from huggingface_hub import snapshot_download

   # Specify the target directory for downloading the model
   target_directory = "models/hf_models/OpenELM-3B-Instruct"

   # Download a snapshot of the model repository
   snapshot_download(
       repo_id="apple/OpenELM-3B-Instruct",
       local_dir=target_directory,
       revision="main",  # Optional: specify a branch, tag, or commit hash
       local_dir_use_symlinks=False  # Set to True if you want symlinks instead of file copies
   )
   ```

2. Run the script to start downloading the model weights:

   ```bash
   python download_openelm.py
   ```

   > **Note:** The download may take around 20 minutes, depending on your internet speed, as the model is large (~3 billion parameters).

### Converting and Quantizing the Model

Once the model is downloaded, convert it to the GGUF format and quantize it for inference:

1. **Convert the model to GGUF format:**

   ```bash
   mkdir -p models/gguf_models

   python llama.cpp/convert_hf_to_gguf.py models/hf_models/OpenELM-3B-Instruct/ \
       --outfile models/gguf_models/OpenELM-3B-Instruct-f16.gguf \
       --outtype f16
   ```

2. **Quantize the model using llama-quantize:**

   The quantization process reduces the model's size and computational requirements, which can improve inference speed on resource-constrained devices like the Raspberry Pi. Two types of quantization are demonstrated here:
   
   - **Q8_0 Quantization:** This reduces the model to 8-bit precision, balancing model size and performance. It typically offers faster inference with a moderate impact on accuracy.
   - **Q4_0 Quantization:** This reduces the model further to 4-bit precision, providing greater speed and memory savings at the expense of some accuracy.

   For Q8_0 quantization:
   ```bash
   llama.cpp/build/bin/llama-quantize \
       models/gguf_models/OpenELM-3B-Instruct-f16.gguf \
       models/gguf_models/OpenELM-3B-Instruct-q8_0.gguf Q8_0
   ```

   For Q4_0 quantization:
   ```bash
   llama.cpp/build/bin/llama-quantize \
       models/gguf_models/OpenELM-3B-Instruct-f16.gguf \
       models/gguf_models/OpenELM-3B-Instruct-q4_0.gguf Q4_0
   ```

### Prompting the Model

After quantization, you can prompt the different models. For example, to use the Q8_0 quantized model:

```bash
llama.cpp/build/bin/llama-cli \
    -m models/gguf_models/OpenELM-3B-Instruct-q8_0.gguf \
    -p "Could you write a very simple program in C++ to print 'Hello, World!' in less than 10 lines of code?"
```

Follow similar steps for other prompts or to test the Q4_0 model. This setup process on the Raspberry Pi allows you to benchmark LLM inference on an edge device and later compare performance with an ARM Graviton server.




## 2. Setting Up the AWS Arm Graviton Server for Inference Benchmarking

If you have not already followed these steps in Lab 2, start by creating an AWS account and logging in as the root user. Next, set up an EC2 instance using a Graviton processor by following these steps:

1. Search for **EC2** in the AWS Management Console and click on **Launch Instance**.
2. Choose your operating system (AMI). Select **Ubuntu** and toggle the architecture box to **arm64**.
3. For the instance type, select a **m7g.xlarge** and allocate **32 GB** of persistent storage.
4. Launch the instance. Once the instance is created, click **Connect** to get the SSH command. It will look similar to:
   ```bash
   ssh -i "GravitonUbuntu.pem" ubuntu@ec2-13-53-171-129.eu-north-1.compute.amazonaws.com
   ```
5. Use the provided SSH command to connect to the instance.

### Setting Up the Server Environment

Once connected:

1. Clone the repository:
   ```bash
   git clone https://github.com/OliverGrainge/Generative_AI_on_arm.git
   ```
2. Change directory and run the server setup script:
   ```bash
   cd Generative_AI_on_arm
   ./setup_graviton.sh
   ```

### Building the llama.cpp Inference Framework

After setting up the server environment, clone and build the `llama.cpp` framework:

1. **Clone the repository:**
   ```bash
   git clone https://github.com/ggerganov/llama.cpp
   ```
2. **Configure the build:**
   ```bash
   cmake -B llama.cpp/build -S llama.cpp -DCMAKE_CXX_FLAGS="-mcpu=native" -DCMAKE_C_FLAGS="-mcpu=native"
   ```
3. **Compile the framework:**
   ```bash
   cmake --build llama.cpp/build --config Release -j$(nproc) -v
   ```

### Downloading the OpenELM-3B Model Weights

Assuming you've already created the `download_openelm.py` file from Section 1, proceed with:

1. Activate the virtual environment:
   ```bash
   source graviton_env/bin/activate
   ```
2. Run the download script:
   ```bash
   python scripts/download_openelm.py
   ```

### Converting and Quantizing the Model

Once the model has been downloaded:

1. **Convert the model to GGUF format:**
   ```bash
   mkdir models/gguf_models/
   python llama.cpp/convert_hf_to_gguf.py models/hf_models/OpenELM-3B-Instruct/ \
       --outfile models/gguf_models/OpenELM-3B-Instruct-f16.gguf \
       --outtype f16
   ```

2. **Quantize the model:**

   For **Q8_0 quantization:**
   ```bash
   llama.cpp/build/bin/llama-quantize \
       models/gguf_models/OpenELM-3B-Instruct-f16.gguf \
       models/gguf_models/OpenELM-3B-Instruct-q8_0.gguf Q8_0
   ```

   For **Q4_0 quantization:**
   ```bash
   llama.cpp/build/bin/llama-quantize \
       models/gguf_models/OpenELM-3B-Instruct-f16.gguf \
       models/gguf_models/OpenELM-3B-Instruct-q4_0.gguf Q4_0
   ```






Great if you have completed all the steps up untill now you should have a compile llama.cpp inference framework on both your local raspberry pi and a remote Arm Graviton Server with the 3B OpenELM model in floating point 16, Int8 and Int4 precisions. Next to compare how the performance differs between them we should calculate some metrics from them. A good metric to measure is the time to first token. This is the time it takes for the LLM to process the entire input prompt and output a single token. It is critical as it significantly impacts user experience and can for large input prompts be a large computational burden as the llm must process every single token in the input. Another important metric to measure is the generation latency. An LLM autoregressively predicts the next most likely character or token based on the previous context e.g the prompt and any previous responce tokens. It is usually measured in tokens per second and in accelerated by caching the previous activations required to generate the new token. This is called KV Caching as the keys and values of the previous tokens are required to compute the next one. Lucky for us, llama.cpp offers a way to measure this for us, using llama-bench. We can use it with the following command. I will run it locally as an example. 

In [9]:
!llama.cpp/build/bin/llama-bench -m models/gguf_models/OpenELM-3B-Instruct-q4_0.gguf -p 4 -n 2

| model                          |       size |     params | backend    | threads |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | ------: | ------------: | -------------------: |
| openelm 3B Q4_0                |   1.62 GiB |     3.04 B | Metal,BLAS |       8 |           pp4 |        130.44 ± 0.82 |
| openelm 3B Q4_0                |   1.62 GiB |     3.04 B | Metal,BLAS |       8 |           tg2 |         65.69 ± 0.43 |

build: 0ccd7f3 (1)


Your output should look like this:

| model                          |       size |     params | backend    | threads |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | ------: | ------------: | -------------------: |
| openelm 3B Q4_0                |   1.62 GiB |     3.04 B | Metal,BLAS |       8 |           pp4 |        130.44 ± 0.82 |
| openelm 3B Q4_0                |   1.62 GiB |     3.04 B | Metal,BLAS |       8 |           tg2 |         65.69 ± 0.43 |

In this example, there are two types of tests: **pp4** and **tg2**.

- **pp4**: This indicates the prefill stage and measures the time in tokens per second (t/s) it takes to run the forward pass on the input prompt. The number `4` in `pp4` represents the 4 input tokens used, specified by the `-p 4` argument passed to `llama-bench`.

- **tg2**: This indicates the generation stage and measures the time it takes to generate new tokens (t/s). The number `2` in `tg2` represents the number of tokens generated, specified by the `-n 2` argument passed to `llama-bench`.


We can now run a number of experiments. Remember in lab 2, we where able to partition matrix multiplications into different piecies of work, each of them able to be parallelized across KleidiAI microkernels. We can also acchieve this parallelization in the llama.cpp framework by using the --threads argument. Lets setup a benchmark file. 

In [24]:
%%writefile scripts/benchmark_openelm.sh
#!/bin/bash

# benchmark the floating point 16 model and write it into 
mkdir openelm_results
llama.cpp/build/bin/llama-bench -m models/gguf_models/OpenELM-3B-Instruct-f16.gguf -p 12 -n 6 --threads 1,2,3,4 | tee openelm_results/f16_pi_results.txt 
llama.cpp/build/bin/llama-bench -m models/gguf_models/OpenELM-3B-Instruct-q8_0.gguf -p 12 -n 6 --threads 1,2,3,4 | tee openelm_results/q8_pi_results.txt 
llama.cpp/build/bin/llama-bench -m models/gguf_models/OpenELM-3B-Instruct-q4_0.gguf -p 12 -n 6 --threads 1,2,3,4 | tee openelm_results/q4_pi_results.txt 

Overwriting scripts/benchmark_openelm.sh


In [25]:
!chmod +x ./scripts/benchmark_openelm.sh
!./scripts/benchmark_openelm.sh

| model                          |       size |     params | backend    | threads |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | ------: | ------------: | -------------------: |
| openelm 3B F16                 |   5.66 GiB |     3.04 B | Metal,BLAS |       1 |          pp12 |        198.08 ± 0.53 |
| openelm 3B F16                 |   5.66 GiB |     3.04 B | Metal,BLAS |       1 |           tg6 |         25.90 ± 0.06 |
| openelm 3B F16                 |   5.66 GiB |     3.04 B | Metal,BLAS |       2 |          pp12 |        198.36 ± 0.33 |
| openelm 3B F16                 |   5.66 GiB |     3.04 B | Metal,BLAS |       2 |           tg6 |         24.84 ± 0.62 |
| openelm 3B F16                 |   5.66 GiB |     3.04 B | Metal,BLAS |       3 |          pp12 |        194.55 ± 0.33 |
| openelm 3B F16                 |   5.66 GiB |     3.04 B | Metal,BLAS |       3 |           tg6 |         24.94 ± 0.59 |
| openelm 3B F16

now you have got the local results. You should copy and past the following code into a file inside scripts/ on your graviton server you have connected to via ssh. e.g. Copy the cell below: 

In [ ]:
#!/bin/bash
mkdir openelm_results

# benchmark the floating point 16 model and write it into 
mkdir openelm_results
llama.cpp/build/bin/llama-bench -m models/gguf_models/OpenELM-3B-Instruct-f16.gguf -p 12 -n 6 --threads 1,2,3,4 | tee openelm_results/f16_grav_results.txt 
llama.cpp/build/bin/llama-bench -m models/gguf_models/OpenELM-3B-Instruct-q8_0.gguf -p 12 -n 6 --threads 1,2,3,4 | tee openelm_results/q8_grav_results.txt 
llama.cpp/build/bin/llama-bench -m models/gguf_models/OpenELM-3B-Instruct-q4_0.gguf -p 12 -n 6 --threads 1,2,3,4 | tee openelm_results/q4_grav_results.txt 

then open a file with 

nano scripts/benchmark_openelm.sh 

here you can then paste your the contents of the above cell. Then to save and exit the file 
ctrl + o 
followed by 
ctrl + x

then you should make it executable then run it. 

chmod +x ./scripts/benchmark_openelm.sh 
./scripts/benchmark_openelm.sh

once that script has finished we can copy the results back to our pi for further analysis and comparison with the raspberry pi. you can do this with 

In [26]:
!scp -i "/Users/olivergrainge/keys/GravitonUbuntu.pem"  -r ubuntu@ec2-13-53-171-129.eu-north-1.compute.amazonaws.com:Generative_AI_on_arm/openelm_results/* ./openelm_results/

f16_grav_results.txt                          100% 1262    12.8KB/s   00:00    
q4_grav_results.txt                           100% 1262    11.6KB/s   00:00    
q8_grav_results.txt                           100% 1262    12.7KB/s   00:00    


now we have the results for both the local machine and the remote we can parse the results with the script already included in your repository 

In [32]:
!python scripts/parse_results.py openelm_results/f16_pi_results.txt
!python scripts/parse_results.py openelm_results/q8_pi_results.txt
!python scripts/parse_results.py openelm_results/q4_pi_results.txt

!python scripts/parse_results.py openelm_results/f16_grav_results.txt
!python scripts/parse_results.py openelm_results/q8_grav_results.txt
!python scripts/parse_results.py openelm_results/q4_grav_results.txt

Now that should have passed the benchmarking results into a pandas dataframe we can now plot the results. 

In [34]:
import pandas as pd 

df_pi_f16 = pd.read_csv("openelm_results/f16_pi_results.csv")
df_pi_q8 = pd.read_csv("openelm_results/q8_pi_results.csv")
df_pi_q4 = pd.read_csv("openelm_results/q4_pi_results.csv")

df_grav_f16 = pd.read_csv("openelm_results/f16_grav_results.csv")
df_grav_q8 = pd.read_csv("openelm_results/q8_grav_results.csv")
df_grav_q4 = pd.read_csv("openelm_results/q4_grav_results.csv")

,Model,Size (GiB),Params (B),Threads,Test,Speed (t/s),Error (t/s)
0,openelm 3B Q4_0,1.62,3.04,1,pp12,27.33,0.06
1,openelm 3B Q4_0,1.62,3.04,1,tg6,10.23,0.01
2,openelm 3B Q4_0,1.62,3.04,2,pp12,47.95,0.23
3,openelm 3B Q4_0,1.62,3.04,2,tg6,18.24,0.01
4,openelm 3B Q4_0,1.62,3.04,3,pp12,68.97,0.03
5,openelm 3B Q4_0,1.62,3.04,3,tg6,25.99,0.01
6,openelm 3B Q4_0,1.62,3.04,4,pp12,88.43,0.41
7,openelm 3B Q4_0,1.62,3.04,4,tg6,33.18,0.02


Now lets plot the prefill stage latency. 

In [ ]:


filtered_df = df[df['Test'].str.startswith('pp')]